In [ ]:
#part just fr this simulaton
import numpy as np
import math
import time
import matplotlib.pyplot as plt

def convert_velocity2RL(v,w,C_conv_toThymio_right, C_conv_toThymio_left,L,R):
    # convert v in mm/s and w in rad/s to right and left wheel velocities in Thymio units
    vr_rads= (2*v+w*L)/(2*R)
    vl_rads= (2*v-w*L)/(2*R)
    vr=vr_rads*C_conv_toThymio_right
    vl=vl_rads*C_conv_toThymio_left
    return vr, vl

import matplotlib.pyplot as plt
camera_obstructed = 0
# Extended Kalman Filter Code

# A matrix
A_k_minus_1 = np.array([[1.0, 0, 0],
                        [0, 1.0, 0],
                        [0, 0, 1.0]])

# Process noise
process_noise_v_k_minus_1 = np.array([0.01, 0.01, 0.003]) #

# State model noise covariance matrix Q_k
Q_k = np.array([[0.13661, 0, 0],
              [0 ,0.13661, 0],
              [0, 0, 0.00005]])

# Measurement matrix H_k
H_k = np.array([[1.0, 0, 0],
                [0, 1.0, 0],
                [0, 0, 1.0]])

# Sensor measurement noise covariance matrix R_k
R_k= np.array([[0.1, 0, 0],
               [0, 0.1, 0],
               [0, 0, 0.00001]]) #TO BE CHANGED AFTER MEASUREMENTS

# Sensor noise
sensor_noise_w_k = np.array([0, 0, 0]) #zero mean noise

# Function to get B matrix
def getB(yaw, deltak):
    B = np.array([[np.cos(yaw) * deltak, 0],
                  [np.sin(yaw) * deltak, 0],
                  [0, deltak]])
    return B

# Extended Kalman Filter function with a priori state output
def ekf(z_k_observation_vector, state_estimate_k_minus_1, 
        control_vector_k_minus_1, P_k_minus_1, dk):
    # PREDICT
    
    state_estimate_k_priori = A_k_minus_1 @ state_estimate_k_minus_1 + \
                              getB(state_estimate_k_minus_1[2], dk) @ control_vector_k_minus_1 + \
                              process_noise_v_k_minus_1
    state_estimate_k_priori[2] = np.mod(state_estimate_k_priori[2], 2*np.pi)    
             
    # Predict state covariance
    P_k = A_k_minus_1 @ P_k_minus_1 @ A_k_minus_1.T + Q_k
         
    # MEASUREMENT residual
    measurement_residual_y_k = z_k_observation_vector - (H_k @ state_estimate_k_priori + sensor_noise_w_k)
             
    # Measurement residual covariance
    S_k = H_k @ P_k @ H_k.T + R_k
         
    # Kalman gain
    if camera_obstructed == 1:
        K_k = np.zeros((3,3))
    else:
        K_k = P_k @ H_k.T @ np.linalg.pinv(S_k)
         
    # UPDATE state estimate
    state_estimate_k = state_estimate_k_priori + (K_k @ measurement_residual_y_k)
     
    # Update state covariance
    P_k = P_k - (K_k @ H_k @ P_k)
     
    return state_estimate_k_priori, state_estimate_k, P_k


#---------------------exemple3 trajectory for the Thymio to follow---------------------
#  --> normally given by the global planner
N = 6
x_goal = np.linspace(1, 6, N)*100 #mm
y_goal = np.array([0, 2, 2, 3, 1, 0])*100 #mm
print(x_goal,y_goal)
camera_obstructed = 1

In [ ]:
from tdmclient import ClientAsync 
client = ClientAsync() 
node = await client.wait_for_node() 
await node.lock() 
 
activate_functions = True 
 
await node.wait_for_variables()

In [ ]:
import numpy as np

def control_law(state_estimate_k, x_goal, y_goal, theta_goal):

    x = state_estimate_k[0]
    y = state_estimate_k[1]
    theta = state_estimate_k[2]
    
    x_diff = x_goal - x
    y_diff = y_goal - y

    alpha = (math.atan2(y_diff, x_diff)- theta + math.pi) % (2 * math.pi) - math.pi
    beta = (theta_goal - theta - alpha + math.pi) % (2 * math.pi) - math.pi

    v = 55
    w = Kp_alpha * alpha + Kp_beta * beta

    # In this step, the direction of v is flipped when alpha is between pi/2 and pi or between -pi/2 and pi
    if alpha > math.pi / 2 or alpha < -math.pi / 2:
        v = -v

    # The control function returns the linear and angular velocities computed which will then be converted to Thymio 
    # speeds to be input to the motors
    return v,w

In [ ]:
#---------------------exemple3 trajectory for the Thymio to follow---------------------
#  --> normally given by the global planner
N = 6
x_goal = np.linspace(1, 6, N)*100 #mm
y_goal = np.array([1, 1, 3, 3, 5, 5])*100 #mm
print(x_goal,y_goal)
camera_obstructed = 1

#PARAMETERS initialization:
Kv = 100 # constant forward speed in Thymio units
threshold_for_convergence = 50 #mm threshold for saying that Thymio has converged to the next node
N = len(x_goal) #number of nodes in the path
k = 0 #index of the current node in the path
Kp_alpha = 2 #gain for the angle alpha in the control law
Kp_beta = -0.5 #gain for the angle beta in the control law 
not_arrived = True #boolean to check if the robot has arrived to the final goal
P_k_minus_1 = np.array([[0.1, 0, 0],
                            [0, 0.1, 0],
                            [0, 0, 0.1]]) #initial state covariance matrix

#PARAMETERS THAT IN THE FINAL MAIN WOULD HAVE BEEN ALREADY INITIALIZED
Ts = 0.2 #time step
L=104 #mm
R= 20 #mm
state_estimate_k = np.array([0, 0, 0]) #initial state estimate
C_conv_toThymio_left = 67.82946137 #conversion coefficient from mm/s to Thymio units for the left wheel
C_conv_toThymio_right = 67.60908181 #conversion coefficient from mm/s to Thymio units for the right wheel
A= np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) #state transition matrix


#loop
X=[state_estimate_k]
while not_arrived:
    #Which is the temporary goal?
    x_goal_temp = x_goal[k]
    y_goal_temp = y_goal[k]
    print("GOAL",np.array([x_goal_temp, y_goal_temp]))
    if k < N-1:
        theta_goal_temp = (math.atan2(x_goal[k+1]-x_goal[k], y_goal[k+1]-y_goal[k]) + np.pi) % (np.pi/2) - np.pi
    # else if we are at the last node we do not update theta_goal_temp, we assume we want final orientation equal to the
    #orientation in the previous node
    print("theta_goal_temp", theta_goal_temp)

    #Calculate the control law
    v, w = control_law(state_estimate_k, x_goal_temp, y_goal_temp, theta_goal_temp, Kv, Kp_alpha, Kp_beta)
    print("v, w")
    print(v, w)
    vr, vl =  convert_velocity2RL(v,w,C_conv_toThymio_right, C_conv_toThymio_left,L,R)
    #set_motors(vr, vl,node) #function that sets the motors to the calculated velocities #the actual func requires also the node
    vr = int(vr)
    vl = int(vl)
    print("vr, vl")
    print(vr, vl)
    d = {
        "motor.left.target": [vr],
        "motor.right.target": [vl],
    }

    await node.set_variables(d)

    #THIS PART IS JUST FOR THE CONTROLLER SIMULATION

    #---------------JUST FOR CONTROLLER:-----------------
    #B= getB(state_estimate_k[2], Ts)
    #state_estimate_k = A @ state_estimate_k + B @ np.array([v, w]) 
    #X.append(state_estimate_k)
    #print("state_estimate:", state_estimate_k)

    #-----------PART TO UNCOMMENT IN THE FINAL MAIN:-----------
    if camera_obstructed == 1:
        z_k_observation_vector = np.array([state_estimate_k[0], state_estimate_k[1], state_estimate_k[2]])
    [_, state_estimate_k,_] = ekf(z_k_observation_vector, state_estimate_k, [v,w] , P_k_minus_1, Ts) #function that returns the state estimate from the Kalman filter
    print("state_estimate:", state_estimate_k)
    print(k)
    #Check if the robot has arrived to the next node
    if np.linalg.norm(state_estimate_k[:2] - np.array([x_goal_temp, y_goal_temp])) < threshold_for_convergence:
        k = k+1
        if k == N-1:
            not_arrived = False
            print('The robot has arrived to the final goal')
            
            d = {
            "motor.left.target": [0],
            "motor.right.target": [0],
            }

            await node.set_variables(d)
            break

    #wait Ts seconds:
    time.sleep(Ts)





plt.plot(x_goal, y_goal)
plt.show()


